<a href="https://colab.research.google.com/github/wheatfox/google_colab/blob/main/tmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip list

In [ ]:
# 